# Task 1

In [0]:
from pyspark.sql.functions import concat, col, lit 

In [0]:
dbfs_fileStore_prefix = "/FileStore/tables"
prefix = "ontimeperformance"
size = "small"


In [0]:
def clean_column_names(df):
  tempList = [] #Edit01
  for col in df.columns:
      new_name = col.strip()
      new_name = "".join(new_name.split())
      new_name = new_name.replace('.','') 
      tempList.append(new_name) 

  return df.toDF(*tempList) 



In [0]:
def task_1(spark_session, flights_path, aircrafts_path):
    aircrafts_df = spark_session.read.csv(aircrafts_path,
                                        inferSchema=True,
                                        header=True)
    aircrafts_df = aircrafts_df.dropna()
  
    aircrafts = clean_column_names(aircrafts_df)
    
    flights_df = spark_session.read.csv(flights_path,
                                        inferSchema=True,
                                        header=True)
    flights = clean_column_names(flights_df).dropna()
    
    new_flights = flights.join(aircrafts, flights.tail_number == aircrafts.tailnum, how='inner')
    
  

    
    cessna = new_flights.filter(new_flights.manufacturer == "CESSNA") \
                    .orderBy(new_flights.model).groupBy(new_flights.model).count()
    
    cessna = cessna.orderBy('count', ascending=False)
    
    
    
    
    
    
    cessna = cessna.select("*", concat(lit('Cessna '),col("model"),lit(' \t '),col("count")).alias("Result"))
    
    cessna.select("Result").write.mode("overwrite").csv("dbfs:/FileStore/tables/task1")
    

    

In [0]:
task_1(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_{size}.csv", f"{dbfs_fileStore_prefix}/{prefix}_aircrafts.csv")

In [0]:
data_location = "dbfs:/FileStore/tables/task1"

files = dbutils.fs.ls(data_location)
csv_file = [x.path for x in files if x.path.endswith(".csv")][0]
dbutils.fs.mv(csv_file, data_location.rstrip('/') + ".csv")
dbutils.fs.rm(data_location, recurse = True)

Out[11]: True

# See the csv file we stored from a folder

In [0]:
dbutils.fs.ls("dbfs:/FileStore/tables")

Out[12]: [FileInfo(path='dbfs:/FileStore/tables/comments-1.csv', name='comments-1.csv', size=28057586),
 FileInfo(path='dbfs:/FileStore/tables/comments.csv', name='comments.csv', size=28057586),
 FileInfo(path='dbfs:/FileStore/tables/hamlet.txt', name='hamlet.txt', size=210705),
 FileInfo(path='dbfs:/FileStore/tables/ontimeperformance_aircrafts.csv', name='ontimeperformance_aircrafts.csv', size=428796),
 FileInfo(path='dbfs:/FileStore/tables/ontimeperformance_airlines.csv', name='ontimeperformance_airlines.csv', size=19691),
 FileInfo(path='dbfs:/FileStore/tables/ontimeperformance_airports.csv', name='ontimeperformance_airports.csv', size=244438),
 FileInfo(path='dbfs:/FileStore/tables/ontimeperformance_flights_large.csv', name='ontimeperformance_flights_large.csv', size=1050293591),
 FileInfo(path='dbfs:/FileStore/tables/ontimeperformance_flights_medium.csv', name='ontimeperformance_flights_medium.csv', size=105063706),
 FileInfo(path='dbfs:/FileStore/tables/ontimeperformance_flights_small.csv', name='ontimeperformance_flights_small.csv', size=10524263),
 FileInfo(path='dbfs:/FileStore/tables/task1.csv', name='task1.csv', size=157),
 FileInfo(path='dbfs:/FileStore/tables/task2.csv', name='task2.csv', size=413),
 FileInfo(path='dbfs:/FileStore/tables/task2.tsv/', name='task2.tsv/', size=0)]

# Read the csv file we stored

In [0]:
resCsv = sqlContext.read.format("csv").load("/FileStore/tables/task1.csv")
resCsv.show()

+--------------------+
 _c0|
+--------------------+
 Cessna 172M 55|
 Cessna 421C 45|
Cessna 210-5(205)...|
 Cessna 550 27|
 Cessna T210N 19|
 Cessna 150 19|
 Cessna 182P 15|
 Cessna 182A 6|
 Cessna T337G 3|
+--------------------+

#Print the concent of the results out

In [0]:
resCsv.collect()

Out[14]: [Row(_c0='Cessna 172M \t 55'),
 Row(_c0='Cessna 421C \t 45'),
 Row(_c0='Cessna 210-5(205) \t 28'),
 Row(_c0='Cessna 550 \t 27'),
 Row(_c0='Cessna T210N \t 19'),
 Row(_c0='Cessna 150 \t 19'),
 Row(_c0='Cessna 182P \t 15'),
 Row(_c0='Cessna 182A \t 6'),
 Row(_c0='Cessna T337G \t 3')]